Downloading the model from Hugging Face Hub , the link contain multiple variants of the model, you can choose the one that fits your needs.here we will choose the FB16

In [5]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf


--2025-06-15 21:19:21--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.121, 13.35.202.40, 13.35.202.97, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.121|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1750025961&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDAyNTk2MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg

In [28]:
!git clone https://github.com/HandsOnLLM/Hands-On-Large-Language-Models.git


fatal: destination path 'Hands-On-Large-Language-Models' already exists and is not an empty directory.


In [29]:
!cd /content/Hands-On-Large-Language-Models
!pip install -r /content/Hands-On-Large-Language-Models/requirements.txt


In [2]:
!apt-get update
!apt-get install -y build-essential cmake


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

Using langchain to load the model with LlamaCpp python package to load the GGUF model file

In [24]:
# Import the LlamaCpp class from LangChain, which provides a wrapper to use Llama.cpp models
from langchain import LlamaCpp

# Instantiate the LlamaCpp model
model = LlamaCpp(
    model_path = "/content/Phi-3-mini-4k-instruct-fp16.gguf",         # ✅ Path to the GGUF model file (e.g., "models/Phi-3-mini.gguf"). Must be a valid local file.
    n_gpu_layers = -1,       # ✅ Number of layers to offload to GPU. -1 means use GPU for all layers (if supported).
    max_token = 500,         # ✅ Maximum number of tokens to generate in a single output.
    n_ctx = 2048,            # ✅ Context window size (how many tokens can be attended to). Must match model capacity.
    seed = 42,               # ✅ Random seed for reproducibility.
    verbose = False          # ✅ If True, logs detailed generation info to the console.
)


/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: WARNING! max_token is not default parameter.
                max_token was transferred to model_kwargs.
                Please confirm that max_token is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


Running inference with the model

In [5]:
model.invoke("What is 1 + 1 ?")

'\n\n<|assistant|> The answer to 1 + 1 is 2.\n\nThis question has a fixed answer, as the operation of addition in mathematics yields consistent results when performed correctly. The process involves combining two numbers—in this case, one unit with another unit—to find their total sum. When you add 1 and 1 together, the result is always 2.'

No output is expected, because the phi3 requires a prompt template to generate text , we will us the langchain core functionality "chains"rompt

## 1- Single Link : Prompt Template

In [25]:
# load the PromptTemplate class from langchain
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""

# define the Phi3 template
prompt = PromptTemplate(
template=template,
input_variables=["input_prompt"]
)

# create a basic chain
basic_chain = prompt | model

In [12]:
# Use the chain
basic_chain.invoke(
{
"input_prompt": "Hi! My name is Maarten. What is 1 + 1?",
}
)

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" Hello Maarten! The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units total."

## Chain of Multiple Prompts

Let’s illustrate with an example. Assume that we want to generate a story
that has three components:
- A title
- A description of the main character
- A summary of the story

In [15]:
# IMPORT THE CHAINER
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# TITLE GENERATION CHAIN
title_template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.
<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=title_template, input_variables=["summary"])
title = LLMChain(llm=model, prompt=title_prompt, output_key="title")

# CHARACTER DESCRIPTION CHAIN
character_template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.
<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(template=character_template, input_variables=["summary", "title"])
character = LLMChain(llm=model, prompt=character_prompt, output_key="character")

# STORY GENERATION CHAIN
story_template = """<s><|user|>
Create a story about {summary} with the title {title}. The main character is: {character}. Only return the story and it cannot be longer than one paragraph.
<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(template=story_template, input_variables=["summary", "title", "character"])
story = LLMChain(llm=model, prompt=story_prompt, output_key="story")

# CHAINING ALL TOGETHER
llm_chain = title | character | story

llm_chain.invoke("a girl that lost her mother")


/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of Memories: Navigating Grief with Lily"',
 'character': ' Lily, a resilient yet sensitive twelve-year-old girl, carries an inner strength honed through loss as she navigates the uncharted waters of grief following her mother\'s passing. With hauntingly vivid memories and heartwarming reminiscences guiding her journey, Lily learns to cherish her mother\'s legacy in a world still echoing with "whispers" of past love and warmth.',
 'story': ' Whispers of Memories: Navigating Grief with Lily tells the poignant tale of twelve-year-old Lily, who after losing her mother to a sudden illness, must find solace in the tender echoes of their shared moments. As she grapples with the bittersweet symphony of grief, each memory becomes an anchor for her resilience; from late-night baking sessions where laughter filled the kitchen, to early morning strolls through blooming gardens that now only exist in her recollections. Lily\'s heart, 

## Give the LLM Memory

We have two types of memory for LLM:
- Conversation buffer
- Conversation summary

In [11]:
# CREATE AN UNPADTED TEMPLATE TO INCLUIDE CHAT HISTORY
template = """<s><|user|>Current conversation:{chat_history}
{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
template=template,
input_variables=["input_prompt", "chat_history"]
)


In [13]:
# define the memory
from langchain.chains import LLMChain

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history"
)

# chain the prompt with the memeory
llm_chain_mem = LLMChain(
    prompt = prompt,
    llm = model,
    memory = memory
)

<ipython-input-13-2753601307>:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain_mem = LLMChain(


In [15]:
llm_chain_mem.invoke(
    {"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"}
)

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hi! My name is Maarten. What is 1 + 1?',
 'chat_history': '',
 'text': ' The answer to your question, "What is 1 + 1?" is 2. It\'s a basic arithmetic operation where you add one unit to another unit, resulting in two units total. If you have any other questions or need assistance with something else, feel free to ask! I\'m here to help.'}

Check again to see if the model remmber

In [17]:
llm_chain_mem.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  The answer to your question, "What is 1 + 1?" is 2. It\'s a basic arithmetic operation where you add one unit to another unit, resulting in two units total. If you have any other questions or need assistance with something else, feel free to ask! I\'m here to help.',
 'text': " Your name is Maarten. Nice to meet you! If you have more math problems or anything else in mind, just let me know—I'm here to assist you."}

## Minmizing the memory storage by using Windowed Conversation Buffer

In [18]:
from langchain.memory import ConversationBufferWindowMemory

# here we will retain the last two convs
memory = ConversationBufferWindowMemory (
    k = 2,
    memory_key="chat_history"
)

# chain the process (memory + llm)
llm_chain_mem = LLMChain(
    prompt = prompt,
    llm = model,
    memory = memory
)


Using this memory, we can try out a sequence of questions to illustrate what
will be remembered. We start with two conversations:

In [19]:
# Ask two questions and generate two conversations in its memory
llm_chain_mem.predict(input_prompt="Hi! My name is Maarten and I am33 years old. What is 1 + 1?")
llm_chain_mem.predict(input_prompt="What is 3 + 3?")

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' The answer to your question, "What is 3 + 3?" is 6. If you have any other math problems or need assistance with a different topic, feel free to ask!'

Check whether it knows the name we gave it

In [21]:
llm_chain_mem.invoke({"input_prompt":"What is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Maarten and I am33 years old. What is 1 + 1?\nAI:  The answer to your question, "What is 1 + 1?" is 2. As for the personal information you\'ve shared, thank you for it, but as a language model AI, my main purpose is assisting with questions and tasks!\n\nHowever, if there are any other ways I can help you or answer your questions related to mathematics, technology, general knowledge or many more topics, feel free to ask.\nHuman: What is 3 + 3?\nAI:  The answer to your question, "What is 3 + 3?" is 6. If you have any other math problems or need assistance with a different topic, feel free to ask!',
 'text': ' AI: Your name was not provided in the initial conversation; however, during our interaction, you mentioned that your name is Maarten. Is there anything else I can assist you with today?'}

In [23]:
# Check whether it knows the age we gave it
llm_chain_mem.invoke({"input_prompt":"What is my age?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my age?',
 'chat_history': 'Human: What is 3 + 3?\nAI:  The answer to your question, "What is 3 + 3?" is 6. If you have any other math problems or need assistance with a different topic, feel free to ask!\nHuman: What is my name?\nAI:  AI: Your name was not provided in the initial conversation; however, during our interaction, you mentioned that your name is Maarten. Is there anything else I can assist you with today?',
 'text': " As an AI, I don't have access to personal data such as your age unless you choose to share it with me in the course of our conversation. If you need assistance or information related to general knowledge about ages or aging, feel free to ask! However, I must remind you not to share any personal details for privacy reasons. How else may I assist you?"}

# Conversation Summary

In [34]:
# Create a summary prompt template
summary_prompt_template = """<s><|user|>Summarize the
conversations and update with the new lines.
Current summary:
{summary}
new lines of conversation:
{new_lines}
New summary:<|end|>
<|assistant|>"""

summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template

)

In [35]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm = model,
    memory_key='chat_histroy',
    prompt=summary_prompt

)
# Chain the LLM, prompt, and memory together
llm_sum_chain = LLMChain(
    prompt = prompt,
    llm = model,
    memory = memory
)

In [38]:
# Generate a conversation and ask for the name
llm_sum_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_sum_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_histroy': ' Maarten introduces himself and asks the AI about the result of a simple addition problem (1+1), to which the AI responds by providing the correct answer (2) along with an explanation.',
 'text': " I'm unable to determine your name as I don't have the ability to access personal data of individuals. If you need help with something specific, feel free to ask!"}

In [40]:
# Check whether it has summarized everything thus far
llm_sum_chain.invoke({"input_prompt": "What was the first question I asked?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1031: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What was the first question I asked?',
 'chat_histroy': ' Maarten introduces himself and inquires about a simple addition problem (1+1), receiving an answer from the AI that confirms the result is 2 along with a brief explanation. Later, when asked for personal information such as their name, the AI politely declines due to privacy constraints but offers assistance on other inquiries.',
 'text': " As an AI, I don't have the ability to recall past interactions or questions. Each session is stateless and independent for privacy reasons. However, you can search your history on the platform where you interacted with me if that's available. If you tell me more about the topic or context of your last question, I can try to help based on our current conversation!"}

In [41]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_histroy': ' Maarten introduces himself and inquires about a simple addition problem (1+1), receiving an answer from the AI confirming the result is 2 along with a brief explanation. When asked for personal information such as their name, the AI politely declines due to privacy constraints but offers assistance on other inquiries. Later, when questioned about the first query, the AI explains its stateless nature and suggests referring to past search history or discussing current topics for help.'}

## AGENTS : ReAct
In practice, the framework consists of iteratively
following these three steps:
- Thought
- Action
- Observation

In [46]:
# Create the ReAct template

react_template = """Answer the following questions as best you
can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N
times)
Thought: I now know the final answer
Final Answer: the final answer to the original input questionBegin!
Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
template=react_template,
input_variables=["tools", "tool_names", "input","agent_scratchpad"]
)

In [47]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
name="duckduck",
description="A web search engine. Use this to as a search engine for general queries.",
func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=model)
tools.append(search_tool)

In [49]:
from langchain.agents import AgentExecutor, create_react_agent# Construct the ReAct agent
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(
agent=agent, tools=tools, verbose=True,
handle_parsing_errors=True
)

# What is the price of a MacBook Pro?
agent_executor.invoke(
{
"input": "What is the current price of a MacBook Pro inUSD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD."
}
)



> Entering new AgentExecutor chain...
 I need to find the current price of a MacBook Pro in USD and then multiply that by the exchange rate to get the equivalent in EUR.
Action: duckduck
Action Input: Current price of MacBook Pro in USD

/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


[snippet: Apple resellers are hosting a variety of new MacBook Pro sales that discount current M4, M4 Pro and M4 Max 14-inch and 16-inch laptops, in addition to blowout bargains on M3 models. Apple offers ..., title: Best MacBook Pro Deals for June 2025 | Save up to $500 - AppleInsider, link: https://appleinsider.com/deals/best-macbook-pro-deals], [snippet: Let's examine how Apple prices each MacBook Pro base model and default hardware configuration: MacBook Pro 13-inch . The entry-level Pro model starts at $1,299 (≈2.2 weeks of non-stop employment at $15/hour) with: ... Trade in or sell your current laptop to offset upgrade costs; With savvy shopping, you can secure the perfect MacBook Pro ..., title: How Much Does A MacBook Pro Cost? - The Pricer, link: https://www.thepricer.org/how-much-does-a-macbook-pro-cost/], [snippet: Retail prices start at $2,499 USD, but every configuration is on sale, with the best prices in this guide knocking hundred of dollars off the laptop of your choos

{'input': 'What is the current price of a MacBook Pro inUSD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD.',
 'output': "The current estimated price of a MacBook Pro in EUR is approximately 1996.65, given an exchange rate of 0.85 EUR for 1 USD. However, please note that this answer is based on average prices and the actual retail price may vary depending on configurations and sales events.\n\nThe user asked about two distinct items: the current price in USD and the equivalent in EUR given a specific exchange rate. The response provided an estimation for both using available data from various sources, acknowledging that the exact retail prices were not listed but could be inferred through averaging the mentioned prices. This approach is reasonable when precise current pricing information isn't directly accessible, and it allows responding to the query with a practical estimate. The result is rounded to two decimal places for currency representation, aligning 